In [1]:
from config import settings
from src.data import datamodule as dm
from pathlib import Path
from src.model.transformation import FromBuffer , CutPSD
import matplotlib.pyplot as plt
from torch import nn 
from src.data.utils import print_database_overview, load_all_from_db


In [2]:
db_path = Path(settings.path.processed, 'processed.db')
assert db_path.exists(), f"Database not found at {db_path}"
print_database_overview(db_path)
meta_data = load_all_from_db(db_path, 'metadata')
frequency_axis = FromBuffer()(meta_data.loc[0,'frequency_axis'])

SQLite Database Overview
Table: processed_data - id, timestamp, turbine_name, Welch_X, Welch_Y, Welch_Z, RMS_X, RMS_Y, RMS_Z, RollingAverage_X, RollingAverage_Y, RollingAverage_Z, Range_X, Range_Y, Range_Z, Mean_X, Mean_Y, Mean_Z
Table: sqlite_sequence - name, seq
Table: metadata - frequency_axis, sample_rate, window_size, processing_method


In [ ]:

columns = ['Welch']
transform = [nn.Sequential(FromBuffer(), CutPSD(frequency_axis, (0, 5)))]
query_schema = f''' 
SELECT id FROM (
    SELECT Welch_X As Welch, id,timestamp 
    FROM processed_data
    UNION ALL
    SELECT Welch_Y As Welch, id, timestamp  a
    FROM processed_data
    UNION ALL
    SELECT Welch_Z As Welch, id,timestamp  as id
    FROM processed_data
)
WHERE timestamp BETWEEN {settings.norther.training_start} AND {settings.norther.training_end}
'''
data_module = dm.DataModule(db_path, query_schema, columns, transform, batch_size=32, num_workers=4)